# Calculating energy in each EV demand model.

Developed by Siobhan Powell, 2021. 

In [1]:
import os
os.chdir('../..')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import copy
from matplotlib.gridspec import GridSpec
import pickle
from future_grid import FutureDemand

In [3]:
run_year = 2019
nerc_region = 'WECC'
gd_short = pickle.load(open('IntermediateOutputs/generator_data_short_%s_%s.obj'%(nerc_region, str(run_year)), 'rb'))

In [4]:
unit_drops = pd.read_csv('IntermediateOutputs/scheduled_retirements_2019.csv', index_col=0)
additions_df = pd.read_csv('IntermediateOutputs/generator_additions.csv', index_col=0)

def drop_add_generators(future_year, gd_short_init, unit_drops=None, additions_df=None, drop=True, add=True):
    
    gd_short_final = copy.deepcopy(gd_short_init)
    if add:
        added_units = additions_df[additions_df['Year']<future_year]['orispl_unit'].values
        for i, val in enumerate(added_units):
            idx = len(gd_short_final.df)
            loc1 = gd_short_final.df[gd_short_final.df['orispl_unit']==val].index
            gd_short_final.df = pd.concat((gd_short_final.df, gd_short_final.df.loc[loc1]), ignore_index=True)
            gd_short_final.df.loc[idx, 'orispl_unit'] = 'added_'+str(i)
        
    if drop:
        dropped_units = unit_drops[unit_drops['retirement_year']<future_year]['orispl_unit'].values
        gd_short_final.df = gd_short_final.df[~gd_short_final.df['orispl_unit'].isin(dropped_units)].copy(deep=True).reset_index(drop=True)
    
    return gd_short_final

Run calculation using 2030 as a test: 

In [6]:
gd_short_copy = drop_add_generators(2035, gd_short, drop=True, unit_drops=unit_drops, add=True, additions_df=additions_df)
future = FutureDemand(gd_short_copy, year=2035)
future.electrification(scale_vs_given=True)
future.solar_multiplier[2035] = 3.5
future.wind_multiplier[2035] = 3
future.solar()
future.wind()


In [7]:
scens = ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']
res = pd.DataFrame(np.zeros((4,1)), columns=['Total Energy'], index=scens)
for scenario_name in scens:

    timers_extra_info = '_NoTimers' # no control
    scenario_date='20220313'

    folder = '../EVDemandModel_EVScenarios/RunningModel/Outputs/'
    key1 = folder + scenario_name + '_100p' + timers_extra_info + '_WECC_' + scenario_date + '.csv'
    key2 = folder + scenario_name + '_100p' + timers_extra_info + '_weekend_WECC_' + scenario_date + '.csv'
    ev_load = pd.read_csv(key1, index_col=0)  # For 100% of drivers
    if 'Total' not in ev_load.columns:
        ev_load['Total'] = ev_load.sum(axis=1)
    ev_load_weekend = pd.read_csv(key2, index_col=0)
    if 'Total' not in ev_load_weekend.columns:
        ev_load_weekend['Total'] = ev_load_weekend.sum(axis=1)
    pen_level = 1.0
    # apply pen level and convert to MW
    ev_load_add = pen_level * (1/1000) * ev_load['Total'].values
    ev_load_weekend_add = pen_level * (1/1000) * ev_load_weekend['Total'].values


    test = copy.deepcopy(future.demand)
    test['ev_demand'] = 0

    for i in range(365):
        if pd.to_datetime(test.loc[24*i, 'datetime']).weekday() in [0, 1, 2, 3, 4]:
            test.loc[24*i+np.arange(0, 24), 'ev_demand'] += ev_load_add[np.arange(0, 1440, 60)]
        else:
            test.loc[24*i+np.arange(0, 24), 'ev_demand'] += ev_load_weekend_add[np.arange(0, 1440, 60)]
            
    res.loc[scenario_name, 'Total Energy'] = test['ev_demand'].sum()

In [8]:
res['Norm by Max'] = res['Total Energy']/res['Total Energy'].max()
res['Norm by Min'] = res['Total Energy']/res['Total Energy'].min()
res['Norm by Mean'] = res['Total Energy']/res['Total Energy'].mean()

In [9]:
res

,Total Energy,Norm by Max,Norm by Min,Norm by Mean
UniversalHome,8.994325e+07,1.000000,1.039283,1.020955
HighHome,8.920403e+07,0.991781,1.030741,1.012564
LowHome_HighWork,8.654357e+07,0.962202,1.000000,0.982365
LowHome_LowWork,8.669768e+07,0.963915,1.001781,0.984115


In [10]:
print('Percent diff between low and high, over low: ', (res.loc['UniversalHome', 'Total Energy'] - res.loc['LowHome_HighWork', 'Total Energy']) / res.loc['LowHome_HighWork', 'Total Energy'])
print('Percent diff between low and high, over high: ', (res.loc['UniversalHome', 'Total Energy'] - res.loc['LowHome_HighWork', 'Total Energy']) / res.loc['UniversalHome', 'Total Energy'])

Percent diff between low and high, over low:  0.0392828334364587
Percent diff between low and high, over high:  0.03779802010831581
